In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import AutoTokenizer, AutoConfig, pipeline, AutoModelForCausalLM
import torch
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from functools import partial
from repe.mllm_utils import auto_configure_device_map
from repe.mllm_rep_control_contrast_vec import ContrastVecInternlmForCausalLM
from repe.modeling_internlm_xcomposer2 import InternLMXComposer2ForCausalLM

model_name_or_path = "/home/yerong2/models/internlm-xcomposer2-vl-7b"

model = ContrastVecInternlmForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True).eval()
# model = InternLMXComposer2ForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True).eval()

from accelerate import dispatch_model
device_map = auto_configure_device_map(3)
model = dispatch_model(model, device_map=device_map)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model.tokenizer = tokenizer

query = '<ImageHere>Please describe this image'
# image = ['../tiger.jpeg']
image = 'tiger.jpeg'
with torch.autocast(device_type='cuda', dtype=torch.float16):
    # response, _ = model.chat(tokenizer, query, image, do_sample=False, num_beams=3, use_meta=True)
      response, _ = model.chat(tokenizer, query=query, image=image, history=[], do_sample=False)

print(response)